In [ ]:
#THESE SCRIPTS TAKE US FROM RAW DATA TO THE TABLES WE HAVE IN OUR SCHEMA - SEE movie_db_model.png

In [1]:
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
df_title_basics = pd.read_csv('raw_data/title_basics.tsv', sep='\t')
print(list(df_title_basics))
df_title_ratings = pd.read_csv('raw_data/title_ratings.tsv', sep='\t')
print(list(df_title_ratings))

c:\python37\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres']
['tconst', 'averageRating', 'numVotes']


In [3]:
df_title_basics=df_title_basics.merge(df_title_ratings,on='tconst')

In [4]:
#Rename title columns
df_title_basics.columns=['title_id', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes']

#Replace all comman with semicolons to enable database import later
df_title_basics['genres'] = df_title_basics['genres'].str.replace(',', ';')

#Filter out null startYear values
df_title_basics = df_title_basics.loc[~df_title_basics['startYear'].isin(['\\N'])]

#Filter out non films or TV shows
df_title_basics = df_title_basics.loc[~df_title_basics['titleType'].isin(['video', 'videoGame', 'tvEpisode'])]

#Change votes and rating to numeric data types
df_title_basics['numVotes'] = df_title_basics['numVotes'].astype(str).astype(int)
df_title_basics['averageRating'] = df_title_basics['averageRating'].astype(str).astype(float)

#Sort by number of votes
df_title_basics = df_title_basics.sort_values('numVotes', ascending=False)

#then descending by rating
df_title_basics = df_title_basics.sort_values('averageRating', ascending=False)

#Drop duplicates
df_title_basics = df_title_basics.drop_duplicates(subset=['primaryTitle', 'startYear'])

#Cut out all but the top 10000 records
df_title_basics = df_title_basics.head(5000)

#Then output to csv
print(df_title_basics.head())
df_title_basics.to_csv('app_data/title.csv', index=False)

         title_id titleType                             primaryTitle  \
571839  tt1975866     short                              Cuddle Club   
179685  tt0323536     short                             The Outlands   
617109  tt2356920     short                   Laugh! Paws! Be Happy!   
881901  tt8713710   tvMovie  Hill Bill, Mind Noodler - Live in Vegas   
677873  tt3360332     short          Spiti esoteriko: House Internal   

                                  originalTitle  isAdult startYear endYear  \
571839                              Cuddle Club        0      2011      \N   
179685                             The Outlands        0      2002      \N   
617109                   Laugh! Paws! Be Happy!        0      2012      \N   
881901  Hill Bill, Mind Noodler - Live in Vegas        0      2018      \N   
677873          Spiti esoteriko: House Internal        0      2014      \N   

       runtimeMinutes        genres  averageRating  numVotes  
571839             \N         Short

In [5]:
#print column headers for manual population in MySQL workbench
print(list(df_title_basics))

['title_id', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes']
